**NOTE: This notebook is written for the Google Colab platform, which provides free hardware acceleration. However it can also be run (possibly with minor modifications) as a standard Jupyter notebook, using a local GPU.**

In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install skorch

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import numpy as np
from PIL import Image
from torchvision import models
from torchvision import transforms
from skorch import NeuralNetClassifier, NeuralNet
from skimage.transform import resize
import matplotlib.pyplot as plt
import torch

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
!mkdir -p data
!wget -nv -nc -O data/lion.png https://www.dropbox.com/s/djnjkz456tbgfnk/lion.png?dl=1
!wget -nv -nc -O data/imagenet_classes https://www.dropbox.com/s/ma25i7w3jpqex2a/imagenet_classes?dl=1

In [ ]:
#@title -- Auxiliary Functions -- { display-mode: "form" }
device = "cuda" if torch.cuda.is_available() else "cpu"

with open("data/imagenet_classes", "r") as file:
    classes = [c[:-1] for c in file.readlines()]
    
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

unnormalize = transforms.Normalize(
    mean=[-tm/sm for tm, sm in zip(normalize.mean, normalize.std)],
    std=[1.0/ts for ts in normalize.std]
)

transform = transforms.Compose([
    transforms.ToTensor(),
    normalize
])

def preproc_image(img, device=device):
    img = resize(img[:, :, :3], (224, 224))
    img_t = transform(img)
    batch_t = torch.unsqueeze(img_t, 0)
    return batch_t.to(device)

def deproc_image(img_prep):
    img = unnormalize(
        img_prep[0].to('cpu')
    ).detach().numpy().transpose((1, 2, 0))
    return np.minimum(np.maximum(img, 0.0), 1.0)

def decode_proba(proba, top=5):
    proba = proba.ravel()
    ind = np.argsort(proba)
    
    for c in reversed(ind[-top:]):
        print("{}:\t{} ({})".format(
            np.array2string(proba[c], precision=5,
                            suppress_small=False),
            classes[c], c))

# Adversarial Examples

This notebook shows one relatively simple method for generating adversarial examples.

Let us start by loading the 50-layer ResNet architecture pretrained on ImageNet. The network expects 224x224 images at its input and it is able to classify them into 1000 classes (their list is in file data/classes and will also be displayed in the code below).

In [ ]:
module = models.resnet50(pretrained=True)
num_classes = 1000

We wrap the ResNet in our usual ``skorch`` wrapper for easy inference.

In [ ]:
net = NeuralNetClassifier(
    torch.nn.Sequential(
        module,
        torch.nn.Softmax(dim=-1)
    ),
    device=device,
)
net.initialize();

## Parameters

We select the target class here: i.e. the class that we will try to get our image to be misclassifed into.

In [ ]:
# target_class = 231 # collie
# target_class = 413 # assault rifle
# target_class = 847 # tank
target_class = 409 # analog clock

To get the list of all the classes uncomment and run the following cell.

In [ ]:
# for ic, c in enumerate(classes):
#     print("{}:\t{}".format(ic, c))

## Loading and Preprocessing the Original Image

Next we are going to load and display the original image.

In [ ]:
img = plt.imread("data/lion.png")
plt.imshow(img); plt.axis('off');

 We will apply the preprocessing that our pretrained neural net expects using function ``preproc_image``. We will then run the preprocessed image through the net and display the top-5 predictions.

In [ ]:
img_t = preproc_image(img)
proba = net.predict_proba(img_t)
decode_proba(proba)

## Constructing the Loss Function

Our next step will be to construct the loss function that we are going to minimize in order to get our adversarial image. Since it is the adversarial image that we are going to be optimizing, let us create a separate tensor for it. Given that the image is supposed to look like the original image, the sensible thing, of course, is to initialize it by copying the original.

In [ ]:
adv_t = img_t.clone().detach().requires_grad_(True)

Having created our adversarial tensor, we will also wrap the target class in a tensor (of type ``long``) and make sure it is transferred to the correct device.

In [ ]:
target_class_t = torch.as_tensor([target_class], dtype=torch.long).to(device)

When computing the loss, we:
* Run the adversarial example through the network to compute its output ``y``;
* We want the input to be misclassified into ``target_class_t`` so we construct the deception loss as the cross entropy loss with ``y`` and ``target_class_t`` as parameters (let us recall that we also use cross entropy when training a network to predict certain classes);
* We construct the similarity loss as the $L^1$ distance between the adversarial image and the original image;
* We add the two losses up.

In [ ]:
def compute_loss():
    y = module(adv_t)
    deception_loss = torch.nn.functional.cross_entropy(y, target_class_t)
    similarity_loss = torch.nn.functional.l1_loss(adv_t, img_t)
    loss = deception_loss + similarity_loss
    return loss

## The Optimization

We will create an optimizer and provide it with the parameters that it is going to be optimizing: tensor ``adv_t`` in this case.

In [ ]:
optimizer = torch.optim.LBFGS([adv_t])

We define a function that the optimizer is going to run at each step:
* Zero out the gradients from the previous step.
* Compute the loss function.
* Backpropagate the gradients.

The updating of the parameters is, of course, going to be handled by the optimizer itself.

In [ ]:
def opti_step():
    optimizer.zero_grad()
    loss = compute_loss()
    loss.backward()
    return loss

We run the optimizer for a couple of epochs and display the losses.

In [ ]:
for epoch in range(5):
    optimizer.step(opti_step)
    print("Epoch {}; loss {}.".format(epoch, compute_loss().item()))

## Displaying the Adversarial Example

We will process the resulting adversarial example to transform it from a tensor back to a natural image that can be visualized. We also run the adversarial image through our network to make sure that it really does get misclassified. If everything worked out correctly, the image should now get classified as an analog clock or whatever other target class that we chose.

In [ ]:
adv = deproc_image(adv_t)
proba = net.predict_proba(preproc_image(adv))
decode_proba(proba)

We can now plot both: the original image and the adversarial image side by side.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=[10, 6])

axes[0].imshow(img)
axes[0].axis('off')
axes[0].set_title("the original image")

axes[1].imshow(adv)
axes[1].axis('off')
axes[1].set_title("the adversarial example");

The images are going to be visually indistinguishable. To show that they are really not the same and how they differ, we will compute and display their absolute pixel-wise difference (averaging over the colour channels).

In [ ]:
diff = np.abs(img - adv).mean(axis=-1)
plt.imshow(diff, cmap='Greys')
plt.axis('off')
plt.colorbar(label="pixel-wise difference (range [0, 1])");

---

## Task 1: A Different Image and Target Class

**Apply the same procedure to a different image and target class.**

Note: New images can be uploaded **directly through the notebook interface** or alternatively using:
```python
from google.colab import files
content_img = files.upload()
filename = list(content_img)[0]
```

---